# HW-3: Solving the Poisson-Boltzman Equation 

What is our goal?

Our goal is to solve the Poisson-Boltzman equation numerically. 

We expect to go throught the following stages:

1. spatial discretization of the linear diffusion operator using second order central finite differences on uniform meshes in 1D and 2D subject to various boundary conditions; 
2. spatial discretization of the non-linear reaction terms by collocation;
3. linearization by computing exact Jacobians (or otherwise);
4. root finding using Newton-type method with accuracy imposed by tolerances (or use-time integration to resolve the non-linearity);

We intend to go through the following four stages 

1. linear model in 1D (x): 1.1 solve symbolically using sympy.dsolve with the ics option to specify the boundary conditions; 1.2 solve by shooting using solve_bvp(); 1.3 add diagonal term to the 1D Poisson matrix and solve resulting linear system. What kind of post-processing is required? 

2. linear model in 2D (x,y): add diagonal term to the 2D Poisson matrix and solve resulting linear system. What kind of post-processing is required?

3. non-linear model in 1D (x): 3.1 solve symbolically using sympy.dsolve with the ics option to specify the boundary conditions; 3.2 solve by shooting using solve_bvp(); 3.3 solve using finite differences and Newton iteration; define a function with input the state u and as output the non-linear residual evaluated in that state. This non-linear residual consists of a linear and a non-linear term. The linear terms involves multiplication with the 1D Poisson matrix. The non-linear term involves a pointwise opoeration on u; define a function with input the state u and as output the Jacobian evaluated in that state. This Jacobian consists of two terms. The first term is the 1D Poisson matrix. The second term is a diagonal matrix with diagonal the derivative of the pointwise function evaluated in u. Define an initial guess for the Newton iteration by solviong the linear model; solve non-linear problem using optimize.newton() using as arguments the above; compare non-linear solve with transient solve; 

4. non-linear model in 2D (x,y):

Domenico expects that the introduction of automatic differentiation to compute the Jacobian is likely to simplify the overall solution procedure and unlock new simulation possibilities. 

Existing Python solver
- [PypKa](https://github.com/mms-fcul/PypKa)
- [PyGBe](https://www.swmath.org/software/17642) uses BEM, not appropriate here? 

## Import Libraries

In [2]:
import numpy as np
print("Succesfully imported %s -- Version: %s"%(np.__name__,np.__version__))
import scipy
print("Succesfully imported %s -- Version: %s"%(scipy.__name__,scipy.__version__))
import matplotlib.pyplot as plt
print("Succesfully imported %s"%plt.__name__)
import pandas as pd
print("Succesfully imported %s -- Version: %s"%(pd.__name__,pd.__version__))
import sympy as sym 
print("Succesfully imported %s -- Version: %s"%(sym.__name__,sym.__version__))
from scipy import optimize
print("Succesfully imported %s"%optimize.__name__)
from scipy.optimize import fsolve
print("Succesfully imported %s"%fsolve.__name__)
from scipy.signal import find_peaks
print("Succesfully imported %s"%find_peaks.__name__)
from scipy.linalg import orth
print("Succesfully imported %s"%orth.__name__)
from scipy.integrate import odeint
print("Succesfully imported %s"%odeint.__name__)

Succesfully imported numpy -- Version: 1.20.1
Succesfully imported scipy -- Version: 1.6.2
Succesfully imported matplotlib.pyplot
Succesfully imported pandas -- Version: 1.2.4
Succesfully imported sympy -- Version: 1.8
Succesfully imported scipy.optimize
Succesfully imported fsolve
Succesfully imported find_peaks
Succesfully imported orth
Succesfully imported odeint


## Section 1/: Introduction and Model Description 

Describe the Poisson-Boltzmann equation and reference solutions. For simplified cases, analytical solutions seem to be available (Ref 15 and Ref 16 on the wiki page [wiki on the Poisson-Boltzmann](https://en.wikipedia.org/wiki/Poisson–Boltzmann_equation)). The equation is related to the Liouville–Bratu–Gelfand equation [wiki on Liouville–Bratu–Gelfand equation](https://en.wikipedia.org/wiki/Liouville–Bratu–Gelfand_equation). An analytical solution of the Bratu equation is given [here problem 8](http://hplgit.github.io/INF5620/doc/pub/._main_nonlin005.html). These analytical solytions are discussed here [Problem 10.1.12 in Boundary Value Problems for Engineers: with MATLAB Solutions](https://books.google.nl/books?id=iiaeDwAAQBAJ&pg=PA433&lpg=PA433&dq=bratubvp.m&source=bl&ots=oVmpqB_tvy&sig=ACfU3U1KJ0SrnkxYMMJMgKSJKOiwXpclBg&hl=en&sa=X&ved=2ahUKEwjcrOvlrt72AhWe_rsIHZJmAHAQ6AF6BAgWEAM#v=onepage&q&f=false) or [in Nonlinear Ordinary Differential Equations: Analytical Approximation and](https://books.google.nl/books?id=87kqDAAAQBAJ&pg=PA183&lpg=PA183&dq=bratubvp.m&source=bl&ots=WJhHaf7nux&sig=ACfU3U3h-JyRZsRNbIjlfXQsGOBKD9kJcg&hl=en&sa=X&ved=2ahUKEwjluoadsd72AhVG16QKHXrhBCQ4ChDoAXoECBIQAw#v=onepage&q&f=false). These analytical solutions provide a reference for the numerical computations. Possibly the analytical solutions can be generated using symbolic computations using the sympy.dsolve() function with the bcs option to specify the boundary condtions. 

An alternative reference solution is [poisson-boltzmann-pham-2007](../other/references/poisson-boltzmann-pham-2007.pdf). 

## Section 2/:  Linear Model in 1D 

Model can be linearized by evaluating the non-linear terms in a assumed previously computed solution (Helmholtz like equation). 

Three solution approaches are available. 

First approach: symbolically using the sympy.dsolve() function with the bcs option to specify the boundary condtions; 

Second approach: numerically using the scipy.integrate.solve_bvp() following the examples on [examples on sol ve_bvp for boundary value problems](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_bvp.html)

Third approach: add diagonal term to Poisson-1D matrix, perform linear system solve and plot solution. Copy-paste 1D-Poisson matrix here from Python intro ANM-2021. 

### Section 1.2/: Construct 1D Diffusion Matrix 

In [1]:
N = 4; h=1/N; h2=h*h; xvec = np.linspace(0,1,N+1);
e = np.ones(N+1); 
A = np.diag(-e[:-1],k=1)+np.diag(2*e)+np.diag(-e[:-1],k=-1); # tridiagonal matrix 
A = (1/h2)*A; 
A[0][0]=1; A[0][1]=0;     # handling left-most boundary condition 
A[-1][-1]=2/h2; A[-1][-2]=-2/h2; # handling right-most Neumann boundary condition 
#A[-1][-1]=1; A[-1][-2]=0; # handling right-most Dirichlet boundary condition
print(A); #print(xvec)

NameError: name 'np' is not defined

## Section 3/: Non-Linear Model in 1D

Extend above to non-linear using exact Jacobian and Newton iteration or time-stepping.  

Three solution approaches are available. 

First approach: symbolically using the sympy.dsolve() function with the bcs option to specify the boundary condtions (as above); 

Second approach: numerically using the scipy.integrate.solve_bvp() following the examples on [examples on sol ve_bvp for boundary value problems](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_bvp.html) (as above) 

Third approach: add non-linear term to Poisson-1D matrix, perform non-linear system solve.

In [18]:
# construct the 1D Poisson matrix: full matrix variant  
def construct_poisson1d(N):
    # print('input N = ',N)
    Np1 = N+1; 
    h = 1/N; h2=h**2; 
    e = np.ones(Np1);
    # A = scipy.sparse.spdiags([-e,2*e,-e],[-1,0,1], Np1,Np1,format='csc');
    A = np.diag(2*e)+np.diag(-e[0:-1],1)+np.diag(-e[0:-1],-1);
    A = (1/h2)*A; 
    A[0,0]=1; A[0,1]=0; A[-1,-1]=1; A[-1,-2]=0; 
    return A;

# construct the load vector
def construct_loadvector1d(N):
    Np1 = N+1; 
    h = 1/N;  
    b = np.zeros((Np1,1));
    b[0] = 1.; 
    return b; 

# define the residual functional 
def residual(x,b):
    v = x - b
    print(v)
    return v 

def residual2(x,A,b):
    # unpack data from argument  
    #A = data[0]; b = data[1]; 
    #(A,b) = data; 
    print(A)
    print(b)
    v = x - b
    v = A@x - b
    print(v)
    return v 

In [19]:
# test call to residual function 
N=3; A = construct_poisson1d(N); b = construct_loadvector1d(N); 
Np1 = N+1; 
x0 = np.ones((Np1,1));
x = residual2(x0,A,b)
print(x)

[[ 1.  0.  0.  0.]
 [-9. 18. -9.  0.]
 [ 0. -9. 18. -9.]
 [ 0.  0.  0.  1.]]
[[1.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [1.]]
[[0.]
 [0.]
 [0.]
 [1.]]


In [20]:
# solve non-linear problem using Newton-type iteration  
N=3; A = construct_poisson1d(N); b = construct_loadvector1d(N); 
Np1 = N+1; 

# set initial guess 
x0 = np.ones((Np1,1));

# solve non-linear problem 
# x = optimize.root(residual2,x0,args=(A,b)) #..this option fails - unclear why 
# x = optimize.fsolve(residual2,x0,args=(A,b)) #..this option fails - unclear why 
x = optimize.newton(residual2,x0,args=(A,b))

# print solution 
print(x)

[[ 1.  0.  0.  0.]
 [-9. 18. -9.  0.]
 [ 0. -9. 18. -9.]
 [ 0.  0.  0.  1.]]
[[1.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [1.]]
[[ 1.  0.  0.  0.]
 [-9. 18. -9.  0.]
 [ 0. -9. 18. -9.]
 [ 0.  0.  0.  1.]]
[[1.]
 [0.]
 [0.]
 [0.]]
[[1.36569988e-05]
 [0.00000000e+00]
 [0.00000000e+00]
 [1.00001366e+00]]
[[ 1.  0.  0.  0.]
 [-9. 18. -9.  0.]
 [ 0. -9. 18. -9.]
 [ 0.  0.  0.  1.]]
[[1.]
 [0.]
 [0.]
 [0.]]
[[0.00000000e+00]
 [6.14564944e-05]
 [9.00006146e+00]
 [0.00000000e+00]]
[[ 1.  0.  0.  0.]
 [-9. 18. -9.  0.]
 [ 0. -9. 18. -9.]
 [ 0.  0.  0.  1.]]
[[1.]
 [0.]
 [0.]
 [0.]]
[[0.00000000e+00]
 [1.22912989e-04]
 [9.00012291e+00]
 [0.00000000e+00]]
[[ 1.  0.  0.  0.]
 [-9. 18. -9.  0.]
 [ 0. -9. 18. -9.]
 [ 0.  0.  0.  1.]]
[[1.]
 [0.]
 [0.]
 [0.]]
[[ 0.]
 [ 9.]
 [-9.]
 [ 0.]]
[[ 1.  0.  0.  0.]
 [-9. 18. -9.  0.]
 [ 0. -9. 18. -9.]
 [ 0.  0.  0.  1.]]
[[1.]
 [0.]
 [0.]
 [0.]]
[[ 0.00000000e+00]
 [ 4.50021510e+00]
 [-6.14585927e-05]
 [ 0.00000000e+00]]
[[ 1.  0.  0.  0.]
 [-9. 18. -9.  0.]


## Section 4/: Linear Model in 2D 

Add diagonal term to Poisson-1D matrix, perform linear system solve and plot solution. Copy-paste 2D-Poisson matrix here from Python intro ANM-2021.

##  Section 5/: Non-Linear Model in 2D 

## References

References
1. [Poisson-Boltzmann Equation](https://en.wikipedia.org/wiki/Poisson–Boltzmann_equation)
2. [wiki on DLVO theory](https://en.wikipedia.org/wiki/DLVO_theory)
3. [Implementation of a Non-Linear Diffusion Equation Ready to Roll](https://rveltz.github.io/BifurcationKit.jl/dev/tutorials/Swift-Hohenberg1d/#d-Swift-Hohenberg-equation-(Automatic)) Possibly better examples do exist